# Preprocessing for Autonomous Transposable Element Prediction

Both Genetic and Epigenetic factors greatly influence the expression of transposable elements. As a result, performing differential expression analysis on transposable elements in cases with highly varient genetic landscapes would provide incredibly noisy (highly false positive) results. This is because TEs found located near expressed genes (exonic or intronic) are likely to be equivalently expressed. As such, if there are differences in the expression of these genes between samples, TEs expression is likely to be differentially expressed in a similar manner. To combat this issue, we can simply eliminate transposable elements that have expression that is highly linearly correlated with local gene expression. In doing so, we eliminate the major confounding variable (local genetic expression) when evaluating resultant functional differences between transposable element expression profiles. the process isolates what we call 'autonomous' transposable element expression.

Assumption
1. Transposable elements that are linearly correlated with the expression of local genes (defined in 'Setting Locality Metric' below)  are simply 'passenger' transcripts that have limited functionality is comparision to the associated genetic expression
2. Collinear expression of local TE(s) and Gene(s) is not coincidental

## Input

TE: Tranposable element per loci expression matrix (in TPM) with samples as columns and individual TE expression as rows.
    ** As shown below **
    Column Names: Sample ID
    Row Names: TE name and Genomic Location of form 'TEname_RelativeGenomicLocation_Chromosome__Start_Stop'
        RelativeGenomicLocation: (intron/intergenic/exonic)

Gene: Gene expression matrix (in TPM) with samples as columns and individual gene expression as rows.
    Column Names: SampleID
    Row Names: ENSEMBL ID

## Import 'Input' Dataframes

In this example I am only evaluating a specific subclass of Tranposable Element L1HS (Young Line-1 Element)

In [94]:
import pandas as pd

L1HS = pd.read_csv('~/PycharmProjects/TransposableElements/Data/tpmL1HS.csv', index_col='Element')
Gene = pd.read_csv('~/PycharmProjects/TransposableElements/Data/TCGAgene.csv', index_col='Gene')
# in my case I needed to isolate ensembl IDs
Gene.index = [x.split('.')[0] for x in Gene.index.values.tolist()]
print(Gene.head())
print(L1HS.head())

                 TCGA-DK-A2I6-01A-12R-A18C-07  TCGA-FD-A6TK-01A-42R-A33J-07  \
ENSG00000000003                          4964                          2049   
ENSG00000000005                             2                             0   
ENSG00000000419                          3610                          3019   
ENSG00000000457                           654                           297   
ENSG00000000460                           533                           292   

                 TCGA-UY-A78L-01A-12R-A33J-07  TCGA-4Z-AA84-01A-11R-A39I-07  \
ENSG00000000003                          4237                           828   
ENSG00000000005                            39                             0   
ENSG00000000419                          3148                          1565   
ENSG00000000457                           403                           467   
ENSG00000000460                          1019                          1131   

                 TCGA-XF-A9SK-01A-11R-A42T-07  TCG

## Setting a locality metric

According to Ribeiro et, al. (https://www.nature.com/articles/s42003-022-03831-w) Genes are considered 'immediately local' if they are <100Kb from their neighbour and 'nearby' if they are <1Mb from their neighbour. However, a specific value for such a concept is very difficult to define. For this analyses I will use the 'immediately local' value.

In [114]:
locality = 1000

1. prepare the TE dataframe for BEDTOOLS alignment
* to identify 'immediately local' genes we also need to add 'locality margins' to our TE locations by subtracting out locality metric from the 'start' and adding the locality metric to 'end' as shown below

format:
    Chr[Z] start[- locality] end[+ locality]
    Chr[Z] start[- locality] end[+ locality]
    Chr[Z] start[- locality] end[+ locality]


In [115]:
elementLocations = []

for element in L1HS.index.values.tolist():
    components = element.split(',')[0].split('_')
    chromosome = 'chr' + str(components[2])
    start = components[4]
    end = components[5]
    elementLocation = [chromosome, int(start) - locality, int(end) + locality,element]
    elementLocations.append(elementLocation)

I will also need to prepare a bed file for my reference genes

to do so I first need to extract genomic locations of these genes based on their ENSEMBL ids
1. Download the appropriate reference genome used for the alignment of your samples in my case this is GRCh38. to do so you simply run: 'pyensembl install --release [genome number] --species homo_sapiens' in your terminal window (GRCh38 is 77)
2. Extract genomic location of gene (chr, start, stop, gene name)

In [116]:
import pyensembl

data = pyensembl.EnsemblRelease(77)

geneLocations = []
try:
    for i, ensembl in enumerate(Gene.index.values.tolist()):
        chromosome = 'chr' + str(data.gene_by_id(ensembl).contig)
        start = str(data.gene_by_id(ensembl).start)
        end = str(data.gene_by_id(ensembl).end)
        gene = str(data.gene_by_id(ensembl).gene_name)
        geneLocation = [chromosome, int(start), int(end), gene]
        geneLocations.append(geneLocation)
except ValueError:
    pass

# perform intersection

In [117]:
import pybedtools

TE = pybedtools.BedTool(elementLocations)
GENE = pybedtools.BedTool(geneLocations)

intersect = TE.intersect(GENE, wo=True)
intersectdf = pd.read_table(intersect.fn, names= ['TEchr', 'TEstart', 'TEend', 'TEname', 'GENEchr', 'GENEstart', 'GENEend', 'GENEname', 'OVERLAP'])

# correlate expression of TE and local gene

1. Although not necessary for small datasets, I am going to create a dictionary 'local genes' as keys and 'local TEs' as values
2.

In [118]:
LocalGeneTEPairs = {}

for pair in intersectdf.iterrows():
    TEname = pair[1][3]
    GENEid = data.gene_ids_of_gene_name(pair[1][7])[0]
    if GENEid in LocalGeneTEPairs.keys():
        LocalGeneTEPairs[GENEid].append(TEname)
    else:
        LocalGeneTEPairs[GENEid] = [TEname]

print(LocalGeneTEPairs)

{'ENSG00000200344': ['L1HS_intron_1__28985834_28986182', 'L1HS_intergenic_10__37995444_38000000'], 'ENSG00000159023': ['L1HS_intron_1__28985834_28986182'], 'ENSG00000142687': ['L1HS_intron_1__35484808_35485000'], 'ENSG00000010803': ['L1HS_intron_1__41207752_41208441'], 'ENSG00000127124': ['L1HS_intron_1__41854524_41855316', 'L1HS_intron_1__41855316_41855560', 'L1HS_intergenic_1__41989724_41990366'], 'ENSG00000185104': ['L1HS_intron_1__50918192_50918966'], 'ENSG00000173406': ['L1HS_intergenic_1__58114947_58115999'], 'ENSG00000224209': ['L1HS_intron_1__63239741_63240562', 'L1HS_intron_1__63240556_63242477'], 'ENSG00000116678': ['L1HS_intron_1__65443957_65444209'], 'ENSG00000172260': ['L1HS_intron_1__71513699_71519742', 'L1HS_intron_1__72110156_72110234'], 'ENSG00000116783': ['L1HS_intron_1__74447378_74448273', 'L1HS_intron_1__74518882_74519571'], 'ENSG00000259030': ['L1HS_intron_1__74447378_74448273', 'L1HS_intron_1__74518882_74519571'], 'ENSG00000117069': ['L1HS_intron_1__77000001_77005

In [121]:
import scipy.stats as stats
for gene in LocalGeneTEPairs.keys():
    for te in LocalGeneTEPairs[gene]:
        print(len(Gene.loc[gene,:]))
        print(len(L1HS.loc[te, :]))
    break
        #print(stats.pearsonr(Gene.loc[gene,:], L1HS.loc[te, :]))

431
40
431
40
